### API

De modellen van HuggingFace kunnen via een *inference api* worden aangesproken. Op deze manier wordt een getraind taalmodel aangesproken zonder dat deze lokaal moet worden opgeslaan. Bij een prototype bespaart dit zowel tijd als memory space.

In [18]:
import configparser
import time, json, requests, csv
from langdetect import detect
from googletrans import Translator
import pandas as pd

dict = {
    'sc':"https://api-inference.huggingface.co/models/haining/scientific_abstract_simplification",
    'bart':"https://api-inference.huggingface.co/models/facebook/bart-large-cnn",
    'bart-sc': "https://api-inference.huggingface.co/models/sambydlo/bart-large-scientific-lay-summarisation",
    't5':"https://api-inference.huggingface.co/models/mrm8488/t5-base-finetuned-summarize-news",
    'kis': "https://api-inference.huggingface.co/models/philippelaban/keep_it_simple",
    'gpt-2':'https://api-inference.huggingface.co/models/gpt2',
    'gpt-2-ft': "https://api-inference.huggingface.co/models/gavin124/gpt2-finetuned-cnn-summarization-v2"
}


import requests

In [10]:
config = configparser.ConfigParser()
config.read('../web-app/config.ini')
global huggingface_api_key
huggingface_api_key = config['huggingface']['api_key']

In [11]:
global google_translate
google_translate = Translator()

In [12]:
def query(payload, API_URL):
    headers = {"Authorization": f"Bearer {huggingface_api_key}"}
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [13]:
def summarize(text, key):
        origin_lang = detect(text)

        if origin_lang in ['fr','nl','de']:
            text = google_translate.translate(text=text,src=origin_lang,dest='en').text
        elif origin_lang not in ['fr','nl','de','en']:
            text = google_translate.translate(text=text,src='en',dest='en').text

        length = len(text)

        API_URL = dict.get(key)
        i = 0
        while True and i < 3:
            output = query({
                "inputs": text,
                "parameters": {
                    "repetition_penalty": 4.0,
                    "max_length": length+10
                }
            }, API_URL)

            response = output
            
            if 'error' not in response:
                break
            else:
                i += 1
                try:
                    time.sleep(float(response.get('estimated_time')))
                except:
                    time.sleep(20)
        
        response = output[0]
                
        if 'generated_text' in output[0]:
            text = output[0].get('generated_text')

        if 'summary_text' in output[0]:
            text = output[0].get('summary_text')

        if origin_lang != 'en':
            text = google_translate.translate(text=text,src="en", dest=origin_lang).text

        return text

In [16]:
sentences = [
    "De gedesoriënteerde amfibie verorbert haar nutritievolle maaltijd.",
    "Desalniettemin werd de implementatie van de voorgestelde maatregelen vertraagd door bureaucratische obstakels.",
    "Ondanks het feit dat de resultaten van het onderzoek veelbelovend waren, waren er nog steeds enkele beperkingen die verder onderzoek vereisten.",
    "De commissie concludeerde dat de huidige regelgeving onvoldoende was om de problemen adequaat aan te pakken.",
    "Het is van cruciaal belang dat er rekening wordt gehouden met de potentiële gevolgen van dergelijke beslissingen op lange termijn.",
    "De complexiteit van de situatie vereiste een zorgvuldige afweging van alle beschikbare opties.",
    "De voortdurende ontwikkelingen in de technologie hebben geleid tot aanzienlijke veranderingen in de manier waarop we communiceren en informatie delen.",
    "Het is belangrijk om te benadrukken dat dergelijke initiatieven alleen succesvol kunnen zijn als er voldoende steun is van alle betrokken partijen.",
    "De analyse toonde aan dat er aanzienlijke verschillen waren tussen de verschillende groepen in termen van hun percepties en attitudes.",
    "Er is een groeiende bezorgdheid over de impact van klimaatverandering op onze planeet en de noodzaak om actie te ondernemen.",
    "Het is essentieel dat we ons blijven inzetten voor het bevorderen van duurzame ontwikkeling en het beschermen van onze natuurlijke hulpbronnen."
]


for i in dict.keys():
    for s in sentences:
        try:
            result = summarize(f"Summarize this in Dutch: {s}", i)
            with open('output.csv', mode='a') as file:
                writer = csv.writer(file)
                writer.writerow([i, s])
        except Exception as e:
            print(e)



In [21]:
df = pd.read_csv('output.csv', encoding = "ISO-8859-1", header=None, names=['model','sentence'])
df.head()

,model,sentence
0,sc,De commissie concludeerde dat de huidige regel...
1,sc,De gedesoriënteerde amfibie verorbert haar nut...
2,sc,Desalniettemin werd de implementatie van de vo...
3,sc,Ondanks het feit dat de resultaten van het ond...
4,sc,De commissie concludeerde dat de huidige regel...


Keuze tussen taalmodellen:

SC en BART-SC scoren subjectief hoger dan de andere modellen. Zij bekijken enkel de gekregen zin. De andere modellen zijn eerder geneigd om extra tekst toe te voegen. Er kan niet achterhaald worden waarom dat deze extra tekst wordt meegegeven.
BART-SC kan bijzaak behouden, terwijl SC sneller de neiging heeft om enkel de kernzaak te behouden in de vereenvoudigde tekst.
Bij de API's moet er expliciet worden aangegeven om welke  transformatie dit gaat. 

Ontwikkelaars kunnen voor algemene samenvattings- en vereenvoudigingstaken gebruik maken van algemene taalmodellen die vrij beschikbaar op HuggingFace of dergelijke platforms terug te vinden zijn. GPT-3 blinkt uit in gepersonaliseerde vereenvoudigings- en samenvattingstaken. Engelstalige prompts die expliciet de uitvoertaal vermelden zijn nauwkeuriger dan Nederlandstalige prompts. 